Напишите функцию get_normalize, которая будет принимать тензор с признаками объектов из какого-то датасета с картинками и будет возвращать поканальное среднее и поканальное стандартное отклонение. Гарантируется, что матрица будет иметь размер [N, C, H, W], где N это количество объектов, C — количество каналов, H, W — размеры изображений. Нужно вернуть кортеж из двух тензоров длины C.

Ваша функция должна иметь следующую сигнатуру def get_normalize(features: torch.Tensor):

In [2]:
import torch

def get_normalize(features: torch.Tensor):
    mean = features.mean(dim=(0, 2, 3))
    std = features.std(dim=(0, 2, 3))
    return mean, std

Напишите функцию get_augmentations, которая будет возвращать готовые аугментации для обучающей выборки и для тестовой выборки. Она должна иметь следующую сигнатуру: def get_augmentations(train: bool = True) -> T.Compose:.

Примените следующие аугментации:

 Измените размер изображения, чтобы его размер был 224 на 224 пикселя (и для обучающей и для тестовой выборки).
 Примените какие-нибудь аугментации из тех, что мы изучили на занятии (только для обучающей).
 Преобразуйте картинку в тензор.
 Примените нормализацию для датасета CIFAR10
Понятно, что изменение размера к 224 на 224 и нормализация для CIFAR10 вместе сочетаются плохо. Поэтому в дальнейшем, когда будете использовать эту функцию для получения аугментаций для вашего нового датасета, не забудьте поменять их на специфичные ему.

In [3]:
from torchvision import transforms as T

def get_augmentations(train: bool = True) -> T.Compose:
    
    means = (0.49139968, 0.48215841, 0.44653091)
    stds = (0.24703223, 0.24348513, 0.26158784)
    
    if train:
        return T.Compose([
            T.Resize((32, 32)),  # Изменение размера изображения
            T.RandomAdjustSharpness(sharpness_factor=2),  # Изменение резкости
            T.RandomHorizontalFlip(),  # Случайное горизонтальное отражение
            T.RandomRotation(15),  # Случайный поворот на ±15 градусов
            T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Изменение цвета
            T.ToTensor(),  # Преобразование в тензор
            T.Normalize(means, stds)  # Нормализация
        ])
    else:
        return T.Compose([
            T.Resize((224, 224)),  # Изменение размера изображения
            T.ToTensor(),  # Преобразование в тензор
            T.Normalize(means, stds)  # Нормализация
        ])


Напишите функцию predict. Она должна принимать на вход нейронную сеть, даталоадер и torch.device. Она должна иметь следующую сигнатуру: def predict(model: nn.Module, loader: DataLoader, device: torch.device):

Внутри функции сделайте следующие шаги:

Создайте пустой список для хранения предсказаний.
Проитерируйтесь по даталоадеру.
На каждой итерации сделайте forward pass для батча, посчитайте классы как аргмакс по выходу нейросети, по логитам, добавьте тензор с предсказаниями в список.
Сделайте конкатенацию всех предсказаний и верните этот тензор длины N, по числу объектов в датасете.
Ваша функция должна возвращать тензор с классами.

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader

@torch.inference_mode()
def predict(model: nn.Module, loader: DataLoader, device: torch.device) -> torch.Tensor:
    predictions = []
    model.eval()
    
    for (x, y) in loader:
        output = model(x)
        predict_classes = output.argmax(dim=1)
        predictions.append(predict_classes)
        
    return torch.cat(predictions)

Напишите функцию predict_tta. Она должна принимать на вход нейронную сеть, даталоадер, torch.device и количество итераций по даталоадеру. Она должна иметь следующую сигнатуру: def predict_tta(model: nn.Module, loader: DataLoader, device: torch.device, iterations: int = 2):.

В этой функции мы применим технику, которую кратко обсуждали на занятии  — Test Time Augmentation. Основная идея заключается в том, чтобы сделать для каждой картинки из тестовой выборки несколько аугментированных вариантов и сделать для них предсказания. Потом эти предсказания усреднить и использовать как обычно. В синтаксисе PyTorch это вырождается в создание тестового датасета со случайными аугментациями (либо как на обучающей выборке, либо чуть более слабыми). После этого нужно проитерироваться по созданному датасету несколько раз и усреднить ответы модели.

Внутри функции сделайте следующие шаги:

Запустите цикл по количеству итераций.
Внутри цикла проитерируйтесь по даталоадеру.
Запишите ответы (не классы, а сырые выходы нейросети) модели в один большой тензор размера [N, C], где C — число классов, а N — количество объектов в датасете (то есть мы должны иметь для каждого объекта вектор из выходов нейросети, логиты).
Сделайте из этих тензоров один огромный тензор размера [N, C, iterations], усредните его по итерациям, чтобы его размер стал [N, C]
Дальше предскажите классы для объектов по этому тензору как аргмакс, верните их из функции.
Ваша функция должна возвращать тензор с классами. Не забудьте переводить модель в режим применения и навешивать декторатор для выключения подсчета градиентов.

In [5]:
import torch
import torch.nn as nn
import torch.optim as Optimizer
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10


In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

@torch.inference_mode()
def predict_tta(model: nn.Module, loader: DataLoader, device: torch.device, iterations: int = 2):
    model.eval()
    all_logits = []
    
    for i in range(iterations):
        logits_iter = []
        
        for images, _ in loader: # так как возвращает кортеж изображения и метки
            output = model(images)
            logits_iter.append(output)
            
        logits_iter = torch.cat(logits_iter, dim=0)
        all_logits.append(logits_iter)

    all_logits = torch.stack(all_logits, dim=-1) # Размерность [N, C, iterations]
    avg_logits = all_logits.mean(dim=-1) # Размерность [N, C]

    predictions_classes = avg_logits.argmax(dim=1) # Выбираем класс с максимальной вероятностью

    return predictions_classes
    

In [7]:
def evaluate(model: nn.Module, data_loader: DataLoader, loss_fn) -> tuple[float, float]:
    model.eval()

    total_loss = 0
    total = 0
    correct = 0

    for x, y in tqdm(data_loader, desc='Evaluate'):
        output = model(x)

        loss = loss_fn(output, y)

        total_loss += loss.item()

        _, y_pred = torch.max(output, 1)
        total += y.size(0)
        correct += (y_pred == y).sum().item()

    total_loss /= len(loader)
    accuracy = correct / total

    return total_loss, accuracy

In [8]:
#!g1.1
from tqdm import tqdm


def train(model: nn.Module, data_loader: DataLoader, optimizer: Optimizer, loss_fn) -> tuple[float, float]:
    model.train()

    train_loss = 0
    total = 0
    correct = 0

    for x, y in tqdm(train_loader, desc='Train'):
        optimizer.zero_grad()

        output = model(x)

        loss = loss_fn(output, y)

        train_loss += loss.item()

        loss.backward()

        optimizer.step()
        
        _, y_pred = torch.max(output, 1)
        total += y.size(0)
        correct += (y_pred == y).sum().item()

    train_loss /= len(train_loader)
    accuracy = correct / total

    return train_loss, accuracy

In [12]:
class FirstModel(nn.Module):
    def __init__(self):
        super(FirstModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2),  # Уменьшение размеров вдвое
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2)  # Еще раз уменьшаем размеры вдвое
        )
        
        # Расчет размерности выхода после сверточных слоев
        self.flatten_size = self._get_flatten_size()

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.flatten_size, 128),  # Указываем рассчитанную размерность
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 10)
        )

    def _get_flatten_size(self):
        # Прогоняем фиктивный тензор через сверточные слои для вычисления выходного размера
        with torch.no_grad():
            x = torch.zeros(1, 3, 32, 32)  # CIFAR10 имеет размер 32x32 с 3 каналами
            x = self.conv_layers(x)
            return x.numel()  # Возвращаем общее количество элементов в выходном тензоре

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [13]:
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

# Датасеты с аугментациями
train_dataset = CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=get_augmentations(train=True)
)

test_dataset = CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=get_augmentations(train=False)
)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
from torch.optim import Adam


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

model = first_model()

optimizer = Adam(model.parameters(), lr=0.0005)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(30):
    train_loss, train_accuracy = train(model, train_loader, optimizer, loss_fn)
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Train Accuracy = {train_accuracy}")

test_loss, test_accuracy = evaluate(model, test_loader, loss_fn)
print(f"Test Loss = {test_loss}, Test Accuracy = {test_accuracy}")

# Предсказания на тестовой выборке с использованием TTA
predictions = predict_tta(model, test_loader, device, iterations=3)

Train: 100%|██████████| 782/782 [00:43<00:00, 17.83it/s]


Epoch 1: Train Loss = 1.5436, Train Accuracy = 0.44338


Train: 100%|██████████| 782/782 [00:43<00:00, 18.04it/s]


Epoch 2: Train Loss = 1.2356, Train Accuracy = 0.56178


Train: 100%|██████████| 782/782 [00:44<00:00, 17.58it/s]


...

Epoch 29: Train Loss = 0.7241, Train Accuracy = 0.74742


Train: 100%|██████████| 782/782 [00:48<00:00, 16.23it/s]


Epoch 30: Train Loss = 0.7196, Train Accuracy = 0.75132


Evaluate:   0%|          | 0/157 [00:02<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x200704 and 4096x128)

In [ ]:
# Сохранение предсказаний
torch.save(predictions, "cifar10_predictions.pt")
print("Predictions saved to cifar10_predictions.pt")

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision import transforms as T

# Функция создания модели
def create_simple_conv_cifar():
    class SimpleConvNet(nn.Module):
        def __init__(self):
            super(SimpleConvNet, self).__init__()
            self.conv_layers = nn.Sequential(
                nn.Conv2d(3, 32, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.BatchNorm2d(32),
                nn.MaxPool2d(2, 2),  # Сжатие до половины
                
                nn.Conv2d(32, 64, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.BatchNorm2d(64),
                nn.MaxPool2d(2, 2)  # Снова сжатие до половины
            )
            
            # Автоматический расчет выходного размера
            self.flatten_size = self._get_flatten_size()
            
            self.fc_layers = nn.Sequential(
                nn.Flatten(),
                nn.Linear(self.flatten_size, 128),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(128, 10)
            )
        
        def _get_flatten_size(self):
            with torch.no_grad():
                x = torch.zeros(1, 3, 32, 32)
                x = self.conv_layers(x)
                return x.numel()

        def forward(self, x):
            x = self.conv_layers(x)
            x = self.fc_layers(x)
            return x

    return SimpleConvNet()

# Аугментации для CIFAR10
def get_augmentations(train=True):
    means = (0.49139968, 0.48215841, 0.44653091)
    stds = (0.24703223, 0.24348513, 0.26158784)
    if train:
        return T.Compose([
            T.RandomHorizontalFlip(),
            T.RandomCrop(32, padding=4),
            T.ToTensor(),
            T.Normalize(means, stds)
        ])
    else:
        return T.Compose([
            T.ToTensor(),
            T.Normalize(means, stds)
        ])

# Обучение
def train(model, loader, optimizer, loss_fn, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return running_loss / len(loader), correct / total

# Оценка модели
def evaluate(model, loader, loss_fn, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return running_loss / len(loader), correct / total

# Функция TTA
@torch.inference_mode()
def predict_tta(model, loader, device, iterations=3):
    model.eval()
    all_logits = []

    for _ in range(iterations):
        logits_iter = []
        for images, _ in loader:
            images = images.to(device)
            outputs = model(images)
            logits_iter.append(outputs.cpu())
        logits_iter = torch.cat(logits_iter, dim=0)
        all_logits.append(logits_iter)

    all_logits = torch.stack(all_logits, dim=-1)  # [N, C, iterations]
    avg_logits = all_logits.mean(dim=-1)  # [N, C]
    predicted_classes = avg_logits.argmax(dim=1)

    return predicted_classes

# Загрузка данных
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=get_augmentations(train=True))
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=get_augmentations(train=False))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

# Установка устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Инициализация модели, оптимизатора, функции потерь
model = create_simple_conv_cifar().to(device)
optimizer = Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

# Обучение модели
for epoch in range(30):
    train_loss, train_accuracy = train(model, train_loader, optimizer, loss_fn, device)
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Train Accuracy = {train_accuracy:.2%}")

    test_loss, test_accuracy = evaluate(model, test_loader, loss_fn, device)
    print(f"Test Loss = {test_loss:.4f}, Test Accuracy = {test_accuracy:.2%}")

# Предсказания на тестовой выборке с использованием TTA
predictions = predict_tta(model, test_loader, device, iterations=5)

# Сохранение предсказаний
torch.save(predictions, "cifar10_predictions.pt")
print("Predictions saved to cifar10_predictions.pt")


100%|██████████| 170498071/170498071 [00:02<00:00, 67475303.37it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch 1: Train Loss = 1.6184, Train Accuracy = 41.08%
Test Loss = 1.1733, Test Accuracy = 57.38%
Epoch 2: Train Loss = 1.3355, Train Accuracy = 51.98%
Test Loss = 1.0158, Test Accuracy = 63.84%
Epoch 3: Train Loss = 1.2235, Train Accuracy = 56.61%
Test Loss = 0.9466, Test Accuracy = 66.88%
Epoch 4: Train Loss = 1.1491, Train Accuracy = 59.31%
Test Loss = 0.8941, Test Accuracy = 68.07%
Epoch 5: Train Loss = 1.0992, Train Accuracy = 61.39%
Test Loss = 0.8351, Test Accuracy = 71.42%
Epoch 6: Train Loss = 1.0599, Train Accuracy = 62.79%
Test Loss = 0.8047, Test Accuracy = 71.97%
Epoch 7: Train Loss = 1.0380, Train Accuracy = 64.06%
Test Loss = 0.7801, Test Accuracy = 72.49%
Epoch 8: Train Loss = 1.0101, Train Accuracy = 64.99%
Test Loss = 0.7838, Test Accuracy = 73.19%
Epoch 9: Train Loss = 0.9858, Train Accuracy = 65.58%
Test Loss = 0.7611, Test Accuracy = 73.55%
Epoch 10: Train Loss = 0.9669, Train A